# In Ensemble2 Part


## Implement Random Forest And Evaluation

======================================================================================

### Importing Modules

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import cross_validation

import numpy as np
import pandas as pd

from collections import Counter

C:\Users\jeonyunsu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Define User's Function : Random Forest

In [2]:
def RandomForest(X,Y,n_estimator, n_subsets, n_features,min_samples_split,X_test):
    
    global pred, model, sample, var, outofbag

    model,sample,outofbag,var=[],[],[],[]
    n=1

    while n<=n_estimator:
        clf=DecisionTreeClassifier(min_samples_split=min_samples_split)
        set_idx=list(np.random.choice(range(0,len(X)),n_subsets,replace=True))
        fea_idx=list(np.random.choice(range(0,len(X[0])),n_features,replace=False))
        sample.append(set_idx);var.append(fea_idx);outofbag.append(np.delete(range(0,len(X)),set_idx,axis=0))
        model.append(clf.fit(X[set_idx][:,fea_idx],Y[set_idx]))
        n+=1

    if len(model)!=1:
        A=[]
        for i in range(0,len(model)):              
            preds=model[i].predict(X_test[:,var[i]])
            A.append(preds)
        
        A=np.array(A)
        pred=[sorted(Counter(A.T[i]), key=Counter(A.T[i]).get, reverse=True)[0] for i in range(0,len(A.T))]
    else:
        pred=model[0].predict(X_test[:,var[0]])       
        
    return model, sample, var

### Importing Datasets and Setting Train/Test Data

In [3]:
frame="segmentation.csv"
Sets=pd.read_table(frame, sep=",")
train, test=cross_validation.train_test_split(Sets,test_size=0.2,stratify=Sets['CLASSES'],random_state=100)
train_X, train_Y=np.array(train)[:,1:],np.array(train)[:,0]
test_X, test_Y=np.array(test)[:,1:],np.array(test)[:,0]

del(train, test,Sets,frame)

### Implement model and Evaluation Performance

In [4]:
min_,sub_=10,130
estimator,features=[1,5,10,20,30,50],[10,15,19]
sub_ACC, sub_model =[], []


for n_estimator in estimator:
    for n_features in features:
        sub_model.append(RandomForest(train_X, train_Y, n_estimator, sub_, n_features, min_,test_X))
        ACC=sum(pred==test_Y)/len(pred)*100
        sub_ACC.append(ACC)
        print("Number of Trees : {} || Number of Features : {} || Accuracy : {}".format(n_estimator, n_features, round(ACC)))

        
best_RFmodel=np.array(sub_model[np.argmax(sub_ACC)]); best_parameter={'Number of Trees':len(best_RFmodel[2,:]),' Number of Features':len(best_RFmodel[2,[0]][0])}
print ("=====================================================================\n",best_parameter)              

Number of Trees : 1 || Number of Features : 10 || Accuracy : 83.0
Number of Trees : 1 || Number of Features : 15 || Accuracy : 81.0
Number of Trees : 1 || Number of Features : 19 || Accuracy : 86.0
Number of Trees : 5 || Number of Features : 10 || Accuracy : 86.0
Number of Trees : 5 || Number of Features : 15 || Accuracy : 86.0
Number of Trees : 5 || Number of Features : 19 || Accuracy : 88.0
Number of Trees : 10 || Number of Features : 10 || Accuracy : 79.0
Number of Trees : 10 || Number of Features : 15 || Accuracy : 88.0
Number of Trees : 10 || Number of Features : 19 || Accuracy : 90.0
Number of Trees : 20 || Number of Features : 10 || Accuracy : 88.0
Number of Trees : 20 || Number of Features : 15 || Accuracy : 86.0
Number of Trees : 20 || Number of Features : 19 || Accuracy : 93.0
Number of Trees : 30 || Number of Features : 10 || Accuracy : 88.0
Number of Trees : 30 || Number of Features : 15 || Accuracy : 90.0
Number of Trees : 30 || Number of Features : 19 || Accuracy : 90.0
N